# CFAR 10

In [ ]:
!pip install tensorflow==2.13.0 keras==2.13.1

In [ ]:
!pip show tensorflow
!pip show keras

In [35]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

2.13.0
2.13.1


In [36]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("Data loaded successfully")

11490434/11490434 [==============================] - 2s 0us/step
Data loaded successfully


# Import Libraries

In [28]:
import numpy as np 
import pandas as pd 
from sklearn.utils.multiclass import unique_labels

import os

In [29]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [30]:
import tensorflow as tf
from keras.applications import ResNet50


'Lastly import the final layers that will be added on top of the base model'
from tensorflow.keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout,UpSampling2D,GlobalAveragePooling2D,Conv2D

# Import dataset


In [31]:
from keras.datasets import cifar10

In [32]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

# Print the dimensions of the datasets to make sure everything's good

In [33]:
print((x_train.shape,y_train.shape))
print((x_test.shape,y_test.shape))

((50000, 32, 32, 3), (50000, 1))
((10000, 32, 32, 3), (10000, 1))


In [34]:
x_train=x_train/255
x_test=x_test/255

## Steps for building a transfer learning model:

* Build the model (Base Model + Flatten + Dense)

* Check model summary

* Initialize Batch Size,Number of Epochs
Compile model

* Fit the model 

* Evaluate the model on test data

# Build the model

In [ ]:
'''For the 2nd base model we will use Resnet 50 and compare the
 performance against the previous one.'''
base_model = ResNet50(include_top=False,weights='imagenet',input_shape=(32, 32, 3))


In [ ]:
base_model.summary()

In [13]:
# for index,layer in enumerate(base_model.layers):
#     if index%2==0:
#         layer.trainable = False
#     else:
#         layer.trainable = True
#     # print(layer)

In [14]:
# # for layer in base_model.layers:
# #     layer.trainable=False

# for layer in base_model.layers[150:175]:
#     layer.trainable=True

In [15]:
for layer in base_model.layers:
    layer.trainable = True

# Set Model

In [16]:
#Since we have already defined Resnet50 as base_model_2, let us build the sequential model.

model = Sequential()

model.add(Conv2D(filters=3, kernel_size=(3,3),input_shape = (32,32,3), activation='relu',padding="same"))

# model.add(UpSampling2D(size=(7, 7)))

model.add(base_model)

model.add(GlobalAveragePooling2D()) # equivalent to model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

# Compile the model 


In [18]:

model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Early Stopping

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
history = model.fit(x_train, y_train, batch_size=2, epochs=20, validation_split=0.2, callbacks=[early_stop])

In [21]:
metrics = pd.DataFrame(model.history.history)

# Metrics

In [ ]:
metrics[['loss', 'val_loss']].plot()

In [ ]:
metrics[['accuracy', 'val_accuracy']].plot()

# Save Model and History

In [ ]:
model.save('cfar10_resnet.h5')

In [ ]:
import pickle

with open('history_resnet.pkl', 'wb') as f:
    pickle.dump(history.history, f)

# Evaluate

In [ ]:
prediction = model.predict(x_test)
predictions = [np.argmax(x, axis=0) for x in prediction]

In [50]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, predictions))

# Analyze

In [52]:
import seaborn as sns

In [25]:
fig_class=[ "airplane", "automobile","bird","cat","deer", "dog","frog","horse","ship","truck"]

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test, predictions),annot=True,fmt='d',xticklabels=fig_class,yticklabels=fig_class)